In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'1ak3muub'

In [3]:
wandb.init(id='1rfauwvw', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 26.5 s, sys: 931 ms, total: 27.4 s
Wall time: 27.5 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


In [9]:
set_lr(42)

In [10]:
model_stem = "11.16"
START_E = 130

In [11]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 138 else 3e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 130

{'trn_control_loss': 0.02234304, 'trn_headings_loss': 0.00116151, 'trn_curvatures_loss': 0.00388131, 'consistency losses/trn_steer_cost': 0.0058081, 'consistency losses/trn_te_loss': 23.32639113, 'aux losses/trn_pitch_loss': 1.42809296, 'aux losses/trn_yaw_loss': 0.37253809, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3019.92962551, 'consistency losses/trn_torque_delta_loss': 29.20846948, 'logistical/obs_consumed_per_second': 189.33070866, 'logistical/obs_generated_per_second': 172.11023622, 'logistical/slowest_runner_obs_per_sec': 10.34645669, 'logistical/data_consumption_ratio': 1.10469958, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.23196983, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 193.1640625, 'logistical/obs_generated_per_second': 178.9765625, 'logistical/slowest_runner_obs_per_sec': 11.5859375, 'logistical/data_consumption_ratio': 1.08237231, 'logistical/manual_tra

{'logistical/obs_consumed_per_second': 197.859375, 'logistical/obs_generated_per_second': 176.609375, 'logistical/slowest_runner_obs_per_sec': 11.6953125, 'logistical/data_consumption_ratio': 1.12098139, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02342796, 'trn_headings_loss': 0.00123139, 'trn_curvatures_loss': 0.00417043, 'consistency losses/trn_steer_cost': 0.00582275, 'consistency losses/trn_te_loss': 24.50165671, 'aux losses/trn_pitch_loss': 1.40402222, 'aux losses/trn_yaw_loss': 0.35980415, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3282.51883698, 'consistency losses/trn_torque_delta_loss': 39.44624243, 'logistical/max_param': 16.26502609, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 193.34375, 'logistical/obs_generated_per_second': 177.8359375, 'logistical/slowest_runner_obs_per_sec': 11.171875, 'logistical/data_consumption_ratio': 1.08861393, 'logistical/manual_train_pause': 0.0, 't

{'logistical/obs_consumed_per_second': 191.671875, 'logistical/obs_generated_per_second': 167.8359375, 'logistical/slowest_runner_obs_per_sec': 10.4609375, 'logistical/data_consumption_ratio': 1.14541998, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02290446, 'trn_headings_loss': 0.00108436, 'trn_curvatures_loss': 0.00427329, 'consistency losses/trn_steer_cost': 0.00607069, 'consistency losses/trn_te_loss': 23.34430818, 'aux losses/trn_pitch_loss': 1.34029388, 'aux losses/trn_yaw_loss': 0.35380411, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 3149.75188637, 'consistency losses/trn_torque_delta_loss': 34.86347667, 'logistical/max_param': 16.24621964, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 193.8515625, 'logistical/obs_generated_per_second': 178.9140625, 'logistical/slowest_runner_obs_per_sec': 13.0390625, 'logistical/data_consumption_ratio': 1.08368516, 'logistical/manual_train_pause': 0.0,

{'logistical/obs_consumed_per_second': 188.5078125, 'logistical/obs_generated_per_second': 170.46875, 'logistical/slowest_runner_obs_per_sec': 10.140625, 'logistical/data_consumption_ratio': 1.10823221, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03217158, 'trn_headings_loss': 0.00111957, 'trn_curvatures_loss': 0.00340226, 'consistency losses/trn_steer_cost': 0.00653636, 'consistency losses/trn_te_loss': 24.10965224, 'aux losses/trn_pitch_loss': 1.52285767, 'aux losses/trn_yaw_loss': 0.34473515, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 2969.93764615, 'consistency losses/trn_torque_delta_loss': 35.71551214, 'logistical/max_param': 16.19986153, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 194.84375, 'logistical/obs_generated_per_second': 180.4375, 'logistical/slowest_runner_obs_per_sec': 10.96875, 'logistical/data_consumption_ratio': 1.081364, 'logistical/manual_train_pause': 0.0, 'trn_cont

{'logistical/obs_consumed_per_second': 190.0, 'logistical/obs_generated_per_second': 174.90625, 'logistical/slowest_runner_obs_per_sec': 11.0546875, 'logistical/data_consumption_ratio': 1.08971667, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03302187, 'trn_headings_loss': 0.00160286, 'trn_curvatures_loss': 0.00552262, 'consistency losses/trn_steer_cost': 0.00778869, 'consistency losses/trn_te_loss': 23.58308555, 'aux losses/trn_pitch_loss': 1.21797943, 'aux losses/trn_yaw_loss': 0.41810036, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 3561.73404884, 'consistency losses/trn_torque_delta_loss': 45.92432247, 'logistical/max_param': 16.22516251, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 195.578125, 'logistical/obs_generated_per_second': 177.6640625, 'logistical/slowest_runner_obs_per_sec': 10.6484375, 'logistical/data_consumption_ratio': 1.10417427, 'logistical/manual_train_pause': 0.0, 'trn_control

...

{'logistical/obs_consumed_per_second': 200.390625, 'logistical/obs_generated_per_second': 179.6171875, 'logistical/slowest_runner_obs_per_sec': 11.0, 'logistical/data_consumption_ratio': 1.11706, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.04434654, 'trn_headings_loss': 0.00155399, 'trn_curvatures_loss': 0.0052495, 'consistency losses/trn_steer_cost': 0.00817166, 'consistency losses/trn_te_loss': 23.02701175, 'aux losses/trn_pitch_loss': 1.38537598, 'aux losses/trn_yaw_loss': 0.27728653, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 2849.75060511, 'consistency losses/trn_torque_delta_loss': 29.50085184, 'logistical/max_param': 16.22730064, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 195.515625, 'logistical/obs_generated_per_second': 184.5625, 'logistical/slowest_runner_obs_per_sec': 12.75, 'logistical/data_consumption_ratio': 1.0589903, 'logistical/manual_train_pause': 0.0, 'trn_control_loss'

{'logistical/obs_consumed_per_second': 202.78125, 'logistical/obs_generated_per_second': 180.8828125, 'logistical/slowest_runner_obs_per_sec': 12.1953125, 'logistical/data_consumption_ratio': 1.1217603, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01403049, 'trn_headings_loss': 0.000812, 'trn_curvatures_loss': 0.00306159, 'consistency losses/trn_steer_cost': 0.00527329, 'consistency losses/trn_te_loss': 23.79543094, 'aux losses/trn_pitch_loss': 1.48630524, 'aux losses/trn_yaw_loss': 0.21872616, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 3056.15017128, 'consistency losses/trn_torque_delta_loss': 27.92757268, 'logistical/max_param': 16.22015762, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 191.2109375, 'logistical/obs_generated_per_second': 181.2890625, 'logistical/slowest_runner_obs_per_sec': 12.0390625, 'logistical/data_consumption_ratio': 1.05577104, 'logistical/manual_train_pause': 0.0, 't

{'logistical/obs_consumed_per_second': 194.34375, 'logistical/obs_generated_per_second': 168.7578125, 'logistical/slowest_runner_obs_per_sec': 10.453125, 'logistical/data_consumption_ratio': 1.15622422, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02762094, 'trn_headings_loss': 0.00152909, 'trn_curvatures_loss': 0.00466738, 'consistency losses/trn_steer_cost': 0.00535066, 'consistency losses/trn_te_loss': 24.81857789, 'aux losses/trn_pitch_loss': 1.30951691, 'aux losses/trn_yaw_loss': 0.30378771, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3255.63614845, 'consistency losses/trn_torque_delta_loss': 34.96012579, 'logistical/max_param': 16.22741127, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 193.390625, 'logistical/obs_generated_per_second': 184.859375, 'logistical/slowest_runner_obs_per_sec': 11.6875, 'logistical/data_consumption_ratio': 1.04673451, 'logistical/manual_train_pause': 0.0, 'trn_

{'logistical/obs_consumed_per_second': 188.8046875, 'logistical/obs_generated_per_second': 185.9453125, 'logistical/slowest_runner_obs_per_sec': 12.8359375, 'logistical/data_consumption_ratio': 1.01769303, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01516813, 'trn_headings_loss': 0.0007819, 'trn_curvatures_loss': 0.00246207, 'consistency losses/trn_steer_cost': 0.00542098, 'consistency losses/trn_te_loss': 23.6510333, 'aux losses/trn_pitch_loss': 1.26046371, 'aux losses/trn_yaw_loss': 0.27431345, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 3262.54041386, 'consistency losses/trn_torque_delta_loss': 36.36700612, 'logistical/max_param': 16.24287796, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 195.96875, 'logistical/obs_generated_per_second': 178.8984375, 'logistical/slowest_runner_obs_per_sec': 10.75, 'logistical/data_consumption_ratio': 1.0981592, 'logistical/manual_train_pause': 0.0, 'trn_co

KeyboardInterrupt: 

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e143.torch")

In [15]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
